# Bike Share Analysis in Python
I learned a lot working on the bike share data using R for the capstone project for my Google Data Analytics certificate. I want to do the same project again but with python to experience the struggle, frustration, and sucess all over again. 

In [2]:
# Setup envionment and import libraries
from matplotlib import pyplot as plt 
import pandas as pd
import seaborn as sns
import numpy as np
import glob
from datetime import datetime

In [4]:
# Read and merge CSV files to dataframe
df = pd.concat(map(pd.read_csv, glob.glob("data/*.csv")))

In [5]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,3564070EEFD12711,electric_bike,2022-04-06 17:42:48,2022-04-06 17:54:36,Paulina St & Howard St,515,University Library (NU),605,42.019135,-87.673532,42.052939,-87.673447,member
1,0B820C7FCF22F489,classic_bike,2022-04-24 19:23:07,2022-04-24 19:43:17,Wentworth Ave & Cermak Rd,13075,Green St & Madison St,TA1307000120,41.853085,-87.631931,41.881892,-87.648789,member
2,89EEEE32293F07FF,classic_bike,2022-04-20 19:29:08,2022-04-20 19:35:16,Halsted St & Polk St,TA1307000121,Green St & Madison St,TA1307000120,41.871840,-87.646640,41.881892,-87.648789,member
3,84D4751AEB31888D,classic_bike,2022-04-22 21:14:06,2022-04-22 21:23:29,Wentworth Ave & Cermak Rd,13075,Delano Ct & Roosevelt Rd,KA1706005007,41.853085,-87.631931,41.867491,-87.632190,casual
4,5664BCF0D1DE7A8B,electric_bike,2022-04-16 15:56:30,2022-04-16 16:02:11,Halsted St & Polk St,TA1307000121,Clinton St & Madison St,TA1305000032,41.871808,-87.646574,41.882242,-87.641066,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634853,8891BA0053ECEC4F,electric_bike,2022-05-27 22:00:02,2022-05-27 22:07:01,Clark St & Newport St,632,NaN,NaN,41.944557,-87.654830,41.920000,-87.650000,member
634854,47D8B5FBCADECFC1,electric_bike,2022-05-15 16:05:39,2022-05-15 16:44:12,Clark St & Newport St,632,NaN,NaN,41.944479,-87.654758,41.920000,-87.760000,member
634855,AA8D16CF38B40703,electric_bike,2022-05-21 10:10:13,2022-05-21 10:26:09,Francisco Ave & Bloomingdale Ave,429,NaN,NaN,41.910000,-87.700000,41.920000,-87.660000,casual
634856,897EBFD44F329E0A,electric_bike,2022-05-12 07:53:58,2022-05-12 08:01:18,Francisco Ave & Bloomingdale Ave,429,NaN,NaN,41.910000,-87.700000,41.900000,-87.690000,member


In [14]:

isodate = df['started_at']
isoformat = '%Y-%m-%d %H:%M:%S'
datetime.strptime(isodate, isoformat)

TypeError: strptime() argument 1 must be str, not Series